In [1]:
import pandas as pd
import numpy as np
import duckdb 
from scipy.sparse import csr_matrix, coo_matrix
from implicit.als import AlternatingLeastSquares

c:\Users\Kutay\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conn = duckdb.connect(database="data/my_database.db")
titles = pd.read_csv("data/movie_titles_new.csv",index_col=0)

In [32]:
print(conn.execute("select * from normalized_rates limit 10").fetchdf())
print((conn.execute("select * from normalized_rates limit 10").fetchdf()).sort_values("NormalizedRate", ascending=False))

   MovieId   UserId       Date  NormalizedRate
0      191   201694 2003-12-10        0.664982
1      191   572279 2005-07-19        1.036926
2      191  2402296 2004-09-12        0.252856
3      191  2573432 2004-02-19       -0.726728
4      191  2350141 2004-01-21        0.458321
5      191  1881794 2003-12-10        0.928037
6      191   846333 2003-12-22        0.518613
7      191  1486244 2004-07-14        1.343912
8      191   390194 2004-12-05        0.731430
9      191  1269373 2003-12-02       -0.208224
   MovieId   UserId       Date  NormalizedRate
7      191  1486244 2004-07-14        1.343912
1      191   572279 2005-07-19        1.036926
5      191  1881794 2003-12-10        0.928037
8      191   390194 2004-12-05        0.731430
0      191   201694 2003-12-10        0.664982
6      191   846333 2003-12-22        0.518613
4      191  2350141 2004-01-21        0.458321
2      191  2402296 2004-09-12        0.252856
9      191  1269373 2003-12-02       -0.208224
3      191  2

In [ ]:
ratings = conn.execute("select Rate, UserId, MovieId from rating").fetch_df()

user_mapping = {id: i for i, id in enumerate(ratings["UserId"].unique())}
movie_mapping = {id: i for i, id in enumerate(ratings["MovieId"].unique())}

ratings["user_idx"] = ratings["UserId"].map(user_mapping)
ratings["movie_idx"] = ratings["MovieId"].map(movie_mapping)

rows = ratings["UserId"].map(user_mapping).values
cols = ratings["MovieId"].map(movie_mapping).values
data = ratings["Rate"].values 

sparse_matrix = csr_matrix((data, (rows, cols)), shape=(len(user_mapping), len(movie_mapping)))

train_matrix = sparse_matrix.copy()  
train_matrix = train_matrix.astype(float)
test_size = int(train_matrix.nnz * 0.01)  
test_indices = np.random.choice(train_matrix.nnz, size=test_size, replace=False)

nonzero_row, nonzero_col = train_matrix.nonzero()

test_size = int(len(nonzero_row) * 0.005)
test_indices = np.random.choice(len(nonzero_row), size=test_size, replace=False)

test_dict = {}
for idx in test_indices:
    row = nonzero_row[idx]
    col = nonzero_col[idx]
    rate = train_matrix[row, col]
    test_dict[idx] = {'row': row, 'col': col, 'rate': rate}

print(f"Test setindeki öğe sayısı: {len(test_dict)}")

train_matrix=train_matrix.tocoo()

train_matrix.data = np.delete(train_matrix.data, test_indices)
train_matrix.row = np.delete(train_matrix.row, test_indices)  
train_matrix.col = np.delete(train_matrix.col, test_indices)

train_matrix = train_matrix.tocsr()

model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=5, random_state=42)
model.fit(train_matrix)
"""
recommended_items, scores = model.recommend(0, train_matrix, N=100, filter_already_liked_items=False)
a= movie_mapping[recommended_items[0]]

print(a, recommended_items)
"""
hit_count = 0
total_test_items = len(test_dict)

for _, entry in test_dict.items():
    row, col = entry["row"], entry["col"]
    
    # Kullanıcı için öneri listesi al
    recommended_items, _ = model.recommend(row, train_matrix, N=10, filter_already_liked_items=True)